In [1]:
import ee # Earth Engine.
from IPython import display
import math
from matplotlib import pyplot
import numpy
#from osgeo import gdal
import tempfile
#import tensorflow as tf
import urllib
import zipfile
# Display preview image in Jupyter Notebook.
from IPython.display import Image, display, HTML

In [2]:
# start the earth engine connection.
ee.Initialize()

# Read in four data collections: 
#    SRTM 30m resolution, 
#    Sentinel-2 data
#    a rough flow accumulation data set on a 90m scale.
SRTM3 = ee.Image('USGS/SRTMGL1_003');
s2 = ee.ImageCollection('COPERNICUS/S2');
fa_90m = ee.Image("users/gena/GlobalHAND/90m-global/fa").convolve(ee.Kernel.gaussian(20, 5));

# A few locations:
#  0 Fort Stockton, 1 Sudan stripes, 2 Somali Haud, 3 McDonnell Range Australia
point_list = [[-103.080477, 31.060309], [28.271546, 11.174898], [47.63, 7.86], [133.794021, -23.4847]]

In [3]:
# Define an MSAVI2 function.  MSAVI2 is a vegetation index.
def addMSAVI2(image):
    # This function adds an 'MSAVI2' band to Sentinel images.
    # Note that the band number associated with a given wavelength
    #   is specific to each satellite, so B8 is near infrared for Sentinel-2
    #   but not for Landsat-7 or Landsat-8.
  return (image.addBands(image.expression(
  '1/2 * ((2*(NIR+1)) - (((2*NIR)+1)**2 - 8*(NIR-RED))**(1/2))', {
      'NIR':image.select('B8'),
      'RED':image.select('B4')}
      ).rename(['MSAVI2'])));

# Define a SAVI function, adding a SAVI band to each Sentinel image.
def addSAVI(image):
  return (image.addBands(image.expression(
  '(1 + L) * float(nir - red)/ (nir + red + L)', {
      'nir':image.select('B8'),
      'red':image.select('B4'),
      'L': 1.0}
      ).rename(['SAVI'])));

# Define a function to create a radius around each location specified above.
# The buffer appears to be measured in meters.
buffersize = 5000;
def addbuffer(f): 
    return f.buffer(buffersize);

# Set up a nice palette for displaying vegetation index.
fmaskPalette2 = ','.join(['FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718',
      '74A901', '66A000', '529400', '3E8601', '207401', '056201',
      '004C00', '023B01', '012E01', '011D01', '011301'])

# Setup a nice palette for displaying data using four gradations.
# Color from colorbrewer2:
# http://colorbrewer2.org/#type=sequential&scheme=Blues&n=4
fmaskPaletteFA = ','.join(['eff3ff','bdd7e7','6baed6', '2171b5'])

In [4]:
# Setup a feature collection for the region of interest

# Choose one of the locations from point_list.
point_index = 0
point_long = point_list[point_index][0]
point_lat = point_list[point_index][1]
point = ee.Geometry.Point(point_long, point_lat);

# Add a buffer to the point.
region = point.buffer(5000).bounds().getInfo()['coordinates'];

# Turn the geometric object into a feature object
feature = ee.Feature(ee.Geometry.Point(point_long,point_lat));

# Turn the feature object into a feature collection
fc = ee.FeatureCollection(feature).map(addbuffer);

In [9]:
# Working version: display SAVI values over a region.
# Specify a date range that will be used to filter the Sentinel-2 data to specific
#   time periods.
startdate = '2015-01-01'
enddate = '2017-04-30'

# Filter the Sentinel-2 imagery by startdate and location to create a filtered collection.
#  All images that overlap the region in fc will be included.
S2 = (s2.filterDate(startdate, enddate).filterBounds(fc));

# print the number of images in the filtered collection
print 'S2size:', S2.size().getInfo()

# S2 = S2.map(addMSAVI2);
# Add the SAVI band to every image from the filtered collection.
S2 = S2.map(addSAVI);

# Select the least cloudy image and the MSAVI2 band.
# S2_0 = ee.Image(S2.sort('CLOUDY_PIXEL_PERCENTAGE', True).first());

# Select the maximum pixel value over the time period for each band.
S2_0 = ee.Image(S2.max());
bands = ['SAVI']

# Filter to just the SAVI band and clip the boundaries to the region of interest.
S2_1 = S2_0.select(bands).clip(fc)

# Display preview image in Jupyter Notebook.
from IPython.display import Image, display, HTML

# Set up the image specifications.  SAVI ranges from 0 to 1.
vis = {
  'min': 0,
  'max': 1,
  'region': region,
  'format': 'png',
  'size': '300',
  'palette': fmaskPalette2};

# Make the image at a URL
th3 = S2_1.getThumbUrl(vis)

# Print the URL where the image can be (temporarily) accessed
print th3

# Also display the image in the notebook.
img_thumb = Image(url=th3)
display(img_thumb)

S2size: 26
https://earthengine.googleapis.com/api/thumb?thumbid=8a5ad469497de811f95409b0837fa928&token=3fe00355f8e5b1547a7641ead008878c


In [6]:
# The CRS varies with band choice.
# EPSG:4326 is a standard one
# EPSG:32613 will be a circle

vis2 = {#'crs': 'EPSG:32613',
  'min': 0,
  'max': 5000,
  'region': region,
  'format': 'png',
  'size': '300',
  'palette': fmaskPaletteFA};

# Turn the flow accumulation dataset into a mask
#  Subtract 10 to recenter it, and multiply by 0.4 to rescale it.
mask = fa_90m.clip(fc).subtract(10).multiply(0.4);

# Make the masked image of the fa_90m data.
th3 = fa_90m.clip(fc).mask(mask).getThumbUrl(vis2)
print th3

# Display the image.
img_thumb = Image(url=th3)
display(img_thumb)

https://earthengine.googleapis.com/api/thumb?thumbid=225a7b0e6ed2846d56e04c3a1c39b522&token=119b3863d28e89d37c2e1697a9b3f28b


In [31]:
# An image that uses 'B4': red, 'B3': green, 'B2': blue as
#  the bands to display.

# Filter to choose the median value of the band over the time period specified before.
S2_2 = ee.Image(S2.median());

# Define the visualization parameters.
visParams = {#'crs': 'EPSG:32613',
  'min': 0,
  'max': 3500,
  'region': region,
  'format': 'png',
  'size': '300',
  #'gamma': [0.95, 1.1, 1],
  'bands': 'B4, B3, B2'};

th4 = S2_2.clip(fc).getThumbUrl(visParams)
print th4

# Display the image.
img_thumb = Image(url=th4)
display(img_thumb)


https://earthengine.googleapis.com/api/thumb?thumbid=db9259705cf61d9cb2cb6212104cebf4&token=2b4bc2d70bfda0d5231f91167a753ea5
